In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import mahalanobis
from scipy.linalg import inv
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

# Step 1: Defining the Matching Framework
# Generate synthetic data for patients with repeated symptom measurements
data = pd.DataFrame({
    'id': range(1, 21),
    'urgency_baseline': np.random.randint(0, 10, 20),
    'urgency_t': np.random.randint(0, 10, 20),
    'frequency_baseline': np.random.randint(0, 10, 20),
    'frequency_t': np.random.randint(0, 10, 20),
    'pain_baseline': np.random.randint(0, 10, 20),
    'pain_t': np.random.randint(0, 10, 20),
    'age': np.random.randint(20, 80, 20),
    'gender': np.random.choice([0, 1], 20),  # 0 = Female, 1 = Male
    'treated': np.random.choice([0, 1], 20)  # 1 = treated, 0 = control
})

# Step 2: Risk Set Matching
# Separate treated and not-yet-treated patients
treated = data[data['treated'] == 1].reset_index(drop=True)
control = data[data['treated'] == 0].reset_index(drop=True)

# Compute covariance matrix and inverse for Mahalanobis distance
cov_matrix = np.cov(data[['urgency_baseline', 'urgency_t', 'frequency_baseline', 'frequency_t', 'pain_baseline', 'pain_t']].T)
cov_inv = inv(cov_matrix)

def compute_distance_matrix(treated, control):
    """Computes Mahalanobis distance matrix between treated and control groups."""
    distances = {}
    for i, t in treated.iterrows():
        for j, c in control.iterrows():
            dist = mahalanobis(t[['urgency_baseline', 'urgency_t', 'frequency_baseline', 'frequency_t', 'pain_baseline', 'pain_t']], 
                               c[['urgency_baseline', 'urgency_t', 'frequency_baseline', 'frequency_t', 'pain_baseline', 'pain_t']], 
                               cov_inv)
            distances[(i, j)] = dist
    return distances

# Compute distance matrix
distance_matrix = compute_distance_matrix(treated, control)

# Step 3: Optimal Matching via Integer Programming
prob = LpProblem("Balanced_Matching", LpMinimize)

# Define decision variables
x = {pair: LpVariable(f"x_{pair[0]}_{pair[1]}", cat='Binary') for pair in distance_matrix.keys()}

# Objective function: Minimize total distance
prob += lpSum(distance_matrix[pair] * x[pair] for pair in distance_matrix.keys())

# Constraints: Each treated patient is matched to one control
for i in range(len(treated)):
    prob += lpSum(x[(i, j)] for j in range(len(control)) if (i, j) in x) == 1

# Each control patient is matched at most once
for j in range(len(control)):
    prob += lpSum(x[(i, j)] for i in range(len(treated)) if (i, j) in x) <= 1

# Step 4: Balancing Covariates
# Enforce balance constraints on binary variables (age, gender)
for attr in ['age', 'gender']:
    treated_sum = lpSum(treated.loc[i, attr] * x[(i, j)] for i, j in x.keys())
    control_sum = lpSum(control.loc[j, attr] * x[(i, j)] for i, j in x.keys())
    prob += treated_sum == control_sum

# Step 5: Ensuring Stability and Sensitivity Analysis
# Solve the optimization problem
prob.solve()

# Extract matched pairs
matches = [(i, j) for (i, j) in x.keys() if x[(i, j)].value() == 1]
print("Matched Pairs:", matches)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/lib/python3.10/dist-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/0795f329c1da444886bb1044c78aa1f1-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/0795f329c1da444886bb1044c78aa1f1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 27 COLUMNS
At line 637 RHS
At line 660 BOUNDS
At line 752 ENDATA
Problem MODEL has 22 rows, 91 columns and 318 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Matched Pairs: [(0, 0), (0, 1), (1, 0), (1, 3), (2, 0), (2, 3), (2, 5), (2, 6), (4, 0), (6, 0), (10, 0), (11, 0), (11, 1), (11, 3), (11, 5), (11, 6), (12, 0)]
